In [60]:
# %load qbo_fub2netcdf.py
#!/usr/bin/python3
'''
Read QBO data from the following file: http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/qbo.dat
'''

import matplotlib as mpl
from matplotlib import dates
import datetime
import xarray as xr
import pandas as pd
import numpy as np
import cftime


def readsing2(nmonth,nyear):
    headerlines = []
    ye = []
    dats = []
    count = 0
    date = []
    #url = "http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/singapore.dat"
    #download(url,"Data/singapore.dat")
    with open('../QBO/qbo_data/singapore.dat') as file:
        for i in range(3):
            headerlines.append(file.readline().strip())
        for year in range(1987,2018):
            if year == nyear:
                data = np.zeros([nmonth,15])*np.nan
            else:
                data = np.zeros([12,15])*np.nan
            ye.append(file.readline().strip())
            file.readline()
            if (year < 1997):
                data = np.zeros([12,15])*np.nan
                for i in range(14):
                    cols = file.readline()
                    cols = cols.strip().split()
                    for j in range(1,13):
                        if i == 1:
                            date.append(datetime.datetime(year, j, 1))
                        data[j-1,i] = float(cols[j])
            else:
                for i in range(15):
                    cols = file.readline()
                    cols = cols.strip().split()
                    for j in range(1,13):
                        if j < nmonth+1 or year < nyear:
                            if i == 1:
                                # print(year)
                                date.append(datetime.datetime(year, j, 1))
                            data[j-1,i] = float(cols[j])
                        # data[j-1,i] = float(cols[j])
            dats.extend([list(i) for i in data])
            file.readline()
            count = count + 1
#    print(np.shape(dats))
#    print(np.shape(np.array(dats)))
    pressure = [100, 90, 80, 70, 60, 50, 45, 40, 35, 30, 25, 20, 15, 12, 10]
    altitude = -7*np.log(np.array(pressure)/1013.15)
    fds = list(mpl.dates.date2num(date))
#    for i in range(np.shape(dats)[0]-np.shape(fds)[0]):
#        fds.append(np.nan)
    fds = np.array(fds)
    # print(np.shape(fds))
    return np.array(dats).T[::-1],fds,pressure,altitude

def readsing():
    #url = "http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/qbo.dat"
    #download(url,"Data/singapore_winds_1953-2017_qbo.dat")
    data = np.genfromtxt('../QBO/qbo_data/qbo.dat',skip_header=9,dtype=['S6','S4','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1'],names=['station','date','p70','n70','p50','n50','p40','n40','p30','n30','p20','n20','p15','n15','p10','n10'],delimiter=[6,4,6,2,5,2,5,2,5,2,5,2,5,2,5,2],filling_values=-999999,missing_values=' ')#
    date = []
    for i in range(len(data)):
        if int(data['date'][i]) > 5000:
            date.append(datetime.datetime.strptime('19'+(data['date'][i]).decode('UTF-8'),'%Y%m'))
        else:
            date.append(datetime.datetime.strptime('20'+(data['date'][i]).decode('UTF-8'),'%Y%m'))

    # print(date[-1])
    fds = mpl.dates.date2num(date)
#
    up = np.array(list([data['p70'],data['p50'],data['p40'],data['p30'],data['p20'],data['p15'],data['p10']]))

    pressure = [70,50,40,30,20,15,10]
    altitude = -7*np.log(np.array(pressure)/1013.15)
#    print(up)
    return up,fds,pressure,altitude

def convert2netcdf(date, u, pressure, fname):
    #date_values = np.array(date).astype('datetime64[M]')
    date_values = np.array(date).astype('datetime64[M]') 

    ds = xr.Dataset({
        'u': (['pressure', 'time'], u, {'_FillValue': np.nan}),
    }, coords={
        'pressure': pressure,
        'time': date.values})

    ds['pressure'].attrs = {'standard_name': 'air_pressure', 'units': 'hPa'}
    ds['u'].attrs = {'standard_name': 'eastward_wind', 'units': 'm/s', 'long_name': 'Zonal Mean Zonal Winds'}
    ds['time'].attrs = {'standard_name': 'time'}
    ds.attrs = {'Conventions': 'CF-1.8', 'title': 'Zonal Mean Zonal Winds', 'history': 'Created by Tobias Kerzenmacher', 'source': 'QBO data source'}
    
    ds.to_netcdf(fname)
    print(ds)
    ds.close()
    
 
#def main():
    # -------------READ----------------------------------------------
up1,fds1,pressure1,altitude1 = readsing()
tnmoth = np.shape(up1)[1] # total number of months
up = np.zeros([15,tnmoth])*np.nan
nmonth = np.shape(up1)[1]%12 # number of months above a whole year
nyear = mpl.dates.num2date(fds1[-1]).year
up2,fds2,pressure,altitude = readsing2(nmonth,nyear)
fds = fds1
fds[-fds2.size:] = fds2
fds = np.array(fds)

up1 = up1*1.
up1[up1 < -10000] = np.nan
up[3,:] = up1[0,:]
up[5,:] = up1[1,:]
up[7,:] = up1[2,:]
up[9,:] = up1[3,:]
up[11,:] = up1[4,]
up[12,:] = up1[5,:]
up[14,:] = up1[6,:]
up[-up2.shape[0]:,-up2.shape[1]:] = up2
ind = ~np.isnan(up)
time,press = np.meshgrid(fds,pressure)
uu = np.array(up[ind])*.1
fname = 'qbo_singapore_fub.nc'
time = mpl.dates.num2date(fds)
date = pd.to_datetime(time)
#da_time = xr.DataArray(time, dims='time', coords={'time': date.values.astype('datetime64[M]')})
convert2netcdf(date,up*0.1,pressure,fname)

<xarray.Dataset>
Dimensions:   (pressure: 15, time: 850)
Coordinates:
  * pressure  (pressure) int64 100 90 80 70 60 50 45 40 35 30 25 20 15 12 10
  * time      (time) datetime64[ns] 1953-01-01 1953-02-01 ... 2017-12-01
Data variables:
    u         (pressure, time) float64 nan nan nan nan ... -33.9 -32.7 -20.3 1.8
Attributes:
    Conventions:  CF-1.8
    title:        Zonal Mean Zonal Winds
    history:      Created by Tobias Kerzenmacher
    source:       QBO data source


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_98888/1997540604.py:92: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  date_values = np.array(date).astype('datetime64[D]')


In [39]:
pd.to_dataarray(time)

AttributeError: module 'pandas' has no attribute 'to_dataarray'

In [40]:
da_time = xr.DataArray({'time': date.values.astype('datetime64[M]')})

In [48]:
da.data

array({'time': array(['1953-01', '1953-02', '1953-03', '1953-04', '1953-05', '1953-06',
       '1953-07', '1953-08', '1953-09', '1953-10', '1953-11', '1953-12',
       '1954-01', '1954-02', '1954-03', '1954-04', '1954-05', '1954-06',
       '1954-07', '1954-08', '1954-09', '1954-10', '1954-11', '1954-12',
       '1955-01', '1955-02', '1955-03', '1955-04', '1955-05', '1955-06',
       '1955-07', '1955-08', '1955-09', '1955-10', '1955-11', '1955-12',
       '1956-01', '1956-02', '1956-03', '1956-04', '1956-05', '1956-06',
       '1956-07', '1956-08', '1956-09', '1956-10', '1956-11', '1956-12',
       '1957-01', '1957-02', '1957-03', '1957-04', '1957-05', '1957-06',
       '1957-07', '1957-08', '1957-09', '1957-10', '1957-11', '1957-12',
       '1958-01', '1958-02', '1958-03', '1958-04', '1958-05', '1958-06',
       '1958-07', '1958-08', '1958-09', '1958-10', '1958-11', '1958-12',
       '1959-01', '1959-02', '1959-03', '1959-04', '1959-05', '1959-06',
       '1959-07', '1959-08', '1959-0